In [1]:
from pathlib import Path
import pandas as pd
import numpy as np

from lpfm.utils.plotting.plot_lossVsTime import LossVsTimePlotter
from lpfm.utils.plotting.base_plotter import calculate_combined_stats, calculate_combined_stats_rollout

pd.set_option('display.max_rows', None)  # Show all rows
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.width', None)  # Auto-detect display width
pd.set_option('display.float_format', lambda x: '%.6f' % x)  # Format float numbers

base_dir = Path("/scratch/zsa8rk/logs")

# Get Mean, Median and STD from loss dataframes
RUNS = [
    # ("fno-s-main", "best_model"),
    # ("fno-m-main", "best_model"),
    # ("ti-main-4-1", "0080"),
    # ("m-main-4-1", "0141"),
    # ("l-main-4-1", "0140"),
    ("xl-main-4-1-a", "0080"),
]

In [ ]:


for run, checkpoint in RUNS:
    eval_dir = base_dir / run / "eval" / checkpoint / "generalization_flipx"
    loss_df = pd.read_csv(eval_dir / "losses.csv", header=0)
    # Example usage:
    # Calculate combined means for different flow types
    flow_patterns = [
        'cylinder_sym_flow_water',
        'cylinder_pipe_flow_water',
        'object_periodic_flow_water',
        'object_sym_flow_air_dt_1',
        'rayleigh_benard',
        'rayleigh_benard_obstacle_dt_1',
        'twophase_flow',
        'shear_flow',
        'euler_multi_quadrants_periodicBC',
        'heated_object_pipe_flow_air',
        'cooled_object_pipe_flow_air',
        'acoustic_scattering_inclusions',
        'supersonic_flow_dt_1',
        'euler_multi_quadrants_openBC_dt_1',
        'open_obj_water_dt_1',

    ]

    combined_means = calculate_combined_stats(loss_df, flow_patterns)

    print(f"Run: {run}, Checkpoint: {checkpoint}")
    display(combined_means)

In [3]:
import yaml
from pathlib import Path

try:
    from yaml import CLoader as Loader
except ImportError:
    from yaml import Loader


def load_config(path: Path) -> dict:
    with open(path, "r") as f:
        config = yaml.load(f, Loader=Loader)
    return config

In [ ]:
from lpfm.run.run_utils import load_stored_model
import torch
from pathlib import Path
from lpfm.data.phys_dataset import PhysicsDataset
from lpfm.model.transformer.model import get_model


model_path = Path("/scratch/zsa8rk/logs/ti-main-4-1/best_model.pth")

config_path = model_path.parent / "config_eval.yaml"
config = load_config(config_path)

model_cp = load_stored_model(model_path, device=torch.device("cpu"),remove_ddp=True)
model_config = config["model"]
model = get_model(model_config)
model.load_state_dict(model_cp["model_state_dict"])
model.eval()

In [5]:
base_path = Path("/scratch/zsa8rk/datasets")
path = base_path / "acoustic_scattering_inclusions" / "data" / "test"

dataset = PhysicsDataset(path, full_trajectory_mode=False, use_normalization=True)

In [6]:
with torch.no_grad():
    x, y = dataset[100]
    x = x.unsqueeze(0)
    y = y.unsqueeze(0)
    pred = model(x)

In [7]:
from lpfm.utils.train_vis import visualize_predictions

visualize_predictions(
    save_path=Path("/home/zsa8rk/Coding/Large-Physics-Foundation-Model/logs/acoustic_scattering_inclusions"),
    inputs=x,
    predictions=pred,
    targets=y,
)